In [ ]:
import re
import unicodedata
import pandas as pd
import numpy as np

from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import SentenceTransformer


def clean_text(text):
    # image link pattern
    img_pattern = r'\b\S+\.(?:png|jpg)\b' 
    # Normalize Unicode characters
    text = unicodedata.normalize('NFKC', text)
    text = text.strip()
    text = re.sub(r'[\r\n\t]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\+', '-', text)
    text = re.sub(r'\s+-\s+', '\n- ', text)
    
    cleaned_text = re.sub(img_pattern, '', text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [ ]:
# Clean the sample input

sample_input = """\r\n\tĐịnh nghĩa\r\n\r\n\r\nXuất tinh sớm là tình trạng mất khả năng kiểm soát và duy trì phản xạ xuất tinh,
gây nên tình trạng xuất tinh mặc dù chưa tới lúc muốn xuất tinh, hậu quả tạo nên sự hụt hẫng cho một hoặc cả hai người nam và nữ.
\r\n\r\n\r\n\tNguyên nhân\r\n\r\n\r\n2.1. Nguyên nhân tâm lý\r\n\r\nĐể có thời gian giao hợp bình thường phải có sự thăng bằng nhịp
nhàng giữa các quá trình hưng phấn và ức chế. Hưng phấn quá mức có thể dẫn đến xuất tinh sớm.\r\n\r\nViệc thủ dâm lâu dài có thể hình thành
nên một phản xạ xuất tinh sớm có điều kiện, từ đó dẫn đến việc xuất tinh sớm trong những lần quan hệ tình dục thực sự về sau.\r\n\r\n2.2.
Nguyên nhân thực thể\r\n\r\n2.2.1. Quá tăng nhạy cảm dương vật\r\n\r\n2.2.2. Các bệnh viêm nhiễm vùng chậu hông\r\n\r\n2.2.3.
Một số bệnh rối loạn nội tiết như: suy sinh dục, mãn dục nam, tăng prolactin máu, … đều gây ảnh hưởng đến chức năng cương và chức năng xuất tinh.
\r\n\r\n2.2.4. Giả thiết về sự tăng nhạy cảm của các thụ quan serotonin \r\n\r\nSự tăng nhạy cảm của các thụ quan serotonin
(thụ quan 5- HT2C và 5-HT1A) ở thần kinh trung ương dẫn đến hiện tượng xuất tinh sớm đã được nhiều tác giả thừa nhận.\r\n\r\n\r\n\tChẩn đoán\r\n\r\n\r\n3.1. Hỏi bệnh\r\n\r\nChẩn đoán xuất tinh sớm chủ yếu dựa vào hỏi bệnh. Các câu hỏi có thể chia thành các nhóm như sau:\r\n\r\n- Câu hỏi để chẩn đoán: \r\n\r\n+ Thời gian xuất tinh là bao lâu? Thời gian xuất tinh (IELT-Intravaginal Ejaculation Latency Time) là thời gian tính từ lúc đưa dương vật vào âm đạo cho tới khi xuất tinh.\r\n\r\n+ Anh có thể trì hoãn xuất tinh được không?\r\n\r\n+ Anh có cảm thấy hụt hẫng do xuất tinh sớm không?\r\n\r\n- Câu hỏi để phân biệt xuất tinh sớm nguyên phát hay thứ phát: \r\n\r\n+ Anh có bị xuất tinh sớm ngay từ lần quan hệ tình dục đầu tiên và trong mọi lần (hoặc hầu như mọi lần) quan hệ tình dục với mọi bạn tình hay là trước đây xuất tinh bình thường, gần đây mới bị trục trặc?\r\n\r\n- Câu hỏi đánh giá rối loạn cương dương: \r\n\r\n+ Dương vật của anh có cương đủ cứng để giao hợp không?\r\n\r\n+ Anh có bị khó khăn để duy trì độ cương dương vật cho tới lúc xuất tinh khi quan hệ tình dục không?\r\n\r\n+ Anh có bao giờ phải cố gắng quan hệ để tránh mất độ cương khi giao hợp không?\r\n\r\n- Câu hỏi đánh giá sự ảnh hưởng lên mối quan hệ: \r\n\r\n+ Bạn tình của anh có thái độ như thế nào khi anh bị xuất tinh sớm?\r\n\r\n+ Xuất tinh sớm có tác động lên mối quan hệ chung của anh với bạn tình hay không?\r\n\r\n- Câu hỏi đánh giá ảnh hưởng lên chất lượng cuộc sống: \r\n\r\n+ Anh có phải tránh quan hệ tình dục do mặc cảm không?\r\n\r\n+ Anh có cảm thấy lo âu, buồn chán vì xuất tinh sớm không?\r\n\r\n- Các câu hỏi khác: \r\n\r\n+ Anh đã bao giờ sử dụng thuốc điều trị xuất tinh sớm chưa?\r\n\r\n+ Anh đã từng lạm dụng hoặc nghiện thuốc gì khác không?\r\n\r\n+ Anh có đang bị bệnh gì khác hay không?\r\n\r\n3.2. Công cụ chẩn đoán\r\n\r\nIELT + Bảng câu hỏi đánh giá xuất tinh sớm PEDT (Premature Ejaculation Diagnostic Tool)\r\n\r\nĐiểm PEDT được tính bằng cách cộng điểm tất cả 5 câu hỏi: Tổng điểm ≤ 8: Không bị xuất tinh sớm.\r\n\r\nTổng điểm = 9 hoặc 10: Có thể đang bị xuất tinh sớm. Tổng điểm ≥ 11: Bị xuất tinh sớm.\r\n\r\n3.3. Khám lâm sàng\r\n\r\nKhám lâm sàng chủ yếu để đánh giá các yếu tố nguy cơ và căn nguyên sinh bệnh, gồm:\r\n\r\n- Khám thực thể: khám bộ phận sinh dục và tuyến tiền liệt,…\r\n\r\n- Khám toàn thân: bệnh tim mạch, rối loạn chuyển hóa, bệnh tâm-thần kinh...\r\n\r\n3.4. Xét nghiệm\r\n\r\nNhìn chung, các xét nghiệm cận lâm sàng ít giá trị cho chẩn đoán xuất tinh sớm. Có thể làm một số xét nghiệm khi cần thiết: xét nghiệm máu (nội tiết tố, sinh hóa,…), xét nghiệm nước tiểu,…\r\n\r\n3.5. Chẩn đoán xác định\r\n\r\nDựa vào kết quả trả lời các câu hỏi, xác định:\r\n\r\n- Bệnh nhân có xuất tinh sớm hay không?\r\n\r\n- Nếu có xuất tinh sớm, thì xuất tinh sớm có phải là bị sau rối loạn cương dương hoặc bệnh lý nào khác không?\r\n\r\n- Xuất tinh sớm thuộc loại nguyên phát (xảy ra ngay từ những lần quan hệ tình dục đầu tiên và trong mọi lần hoặc hầu như mọi lần quan hệ tình dục) hay thứ phát (trước đây xuất tinh bình thường, gần đây mới bị trục trặc).\r\n\r\n\r\n\tPhân tuyến kỹ thuật: theo danh mục kỹ thuật do Bộ Y tế quy định.\r\n\tĐiều trị\r\n\r\n\r\nNguyên tắc điều trị:\r\n\r\n- Ở những bệnh nhân mà xuất tinh sớm do rối loạn cương dương hoặc các bệnh lý khác, nên điều trị rối loạn cương dương trước.\r\n\r\n- Nếu xuất tinh sớm không phải do nguyên nhân khác:\r\n\r\n+ Với xuất tinh sớm nguyên phát: ưu tiên điều trị bằng thuốc trước, phối hợp với điều chỉnh hành vi/liệu pháp tâm lý, có thể kết hợp với tư vấn về mối quan hệ.\r\n\r\n+ Với xuất tinh sớm thứ phát: ưu tiên điều chỉnh hành vi/liệu pháp tâm lý, phối hợp với dùng thuốc, có thể kết hợp với tư vấn về mối quan hệ.\r\n\r\n5.1. Dùng thuốc\r\n\r\n5.1.1. Các thuốc đường uống: Các thuốc có thể dùng để điều trị xuất tinh sớm: \r\n\r\n\r\n\t\r\n\t\t\r\n\t\t\t\r\n\t\t\tCác thuốc đường uống\r\n\t\t\t\r\n\t\t\t\r\n\t\t\tTên thương mại\r\n\t\t\t\r\n\t\t\t\r\n\t\t\tLiều khuyến cáo\r\n\t\t\t\r\n\t\t\r\n\t\t\r\n\t\t\t\r\n\t\t\tThuốc ức chế tái hấp thu serotonin không chọn lọc\r\n\t\t\t\r\n\t\t\r\n\t\t\r\n\t\t\t\r\n\t\t\tClomipramine\r\n\t\t\t\r\n\t\t\t\r\n\t\t\tAnafranil\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t25-50 mg/ngày hoặc 25mg, 4-24 giờ trước khi quan hệ tình dục\r\n\t\t\t\r\n\t\t\r\n\t\t\r\n\t\t\t\r\n\t\t\tThuốc ức chế tái hấp thu serotonin chọn lọc\r\n\t\t\t\r\n\t\t\r\n\t\t\r\n\t\t\t\r\n\t\t\tFluoxetine\r\n\t\t\t\r\n\t\t\t\r\n\t\t\tProzac, Fluozac\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t20-40 mg/ngày hoặc 20mg, 3-4 giờ trước khi quan hệ tình dục\r\n\t\t\t\r\n\t\t\r\n\t\t\r\n\t\t\t\r\n\t\t\tParoxetine\r\n\t\t\t\r\n\t\t\t\r\n\t\t\tPaxil\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t10, 20, 40 mg/ngày hoặc 20mg, 3-4 giờ trước khi quan hệ tình dục\r\n\t\t\t\r\n\t\t\r\n\t\t\r\n\t\t\t\r\n\t\t\tSertraline\r\n\t\t\t\r\n\t\t\t\r\n\t\t\tZoloft\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t25-200 mg/ngày hoặc 50mg, 4-8 giờ trước khi quan hệ tình dục\r\n\t\t\t\r\n\t\t\r\n\t\t\r\n\t\t\t\r\n\t\t\tDapoxetine\r\n\t\t\t\r\n\t\t\t\r\n\t\t\tPriligy\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t30, 60 mg, uống 1-3 giờ trước khi quan hệ tình dục\r\n\t\t\t\r\n\t\t\r\n\t\r\n\r\n\r\nLiều dùng: nên dùng từ liều thấp lên liều cao.\r\n\r\nCho đến nay, Dapoxetine (Priligy) đã được phê chuẩn để điều trị xuất tinh sớm ở trên 50 quốc gia.\r\n\r\n5.1.2. Thuốc gây tê tại chỗ\r\n\r\n- Dùng xylocain, lidocain, prilocaine dạng gel hoặc dạng xịt lên đầu dương vật trước khi giao hợp 20-30 phút. Sau đó rửa sạch rồi mới quan hệ.\r\n\r\n- Nhật Bản, Trung Quốc công bố kem SS điều chế từ thảo dược.\r\n\r\n5.2. Thủ thuật phong bế đám rối thần kinh cùng\r\n\r\nNgười bệnh ở tư thế quỳ, đầu thấp mông cao. Xác định khe khớp giữa S5-S6, sau đó bơm 20ml dung dịch xylocain 0,5% vào khoang cùng. Mỗi tuần 1-2 lần, trong 5-6 tuần liên tục.\r\n\r\n5.3. Phẫu thuật\r\n\r\nCắt bỏ dây thần kinh cảm giác ngoại vi quanh đầu dương vật nhằm mục đích làm giảm sự quá nhạy cảm ở đầu dương vật.\r\n\r\n5.4. Điều trị bằng điều chỉnh hành vi/liệu pháp tâm lý\r\n\r\nTập luyện bằng các phương pháp điều chỉnh hành vi/liệu pháp tâm lý
làm cho người bị xuất tinh sớm có thể kiểm soát được sự xuất tinh, tự tin hơn trong quan hệ tình dục.\r\n"""

cleaned_text = clean_text(sample_input)
print(cleaned_text)

In [ ]:
med_knowledge_df = pd.read_csv("./medical_knowledge - Copy.csv", encoding='utf-16')

In [ ]:
med_knowledge_df['infor'] = med_knowledge_df['infor'].fillna("")
med_knowledge_df['clean_infor'] = med_knowledge_df['infor'].apply(lambda x: clean_text(x))

In [ ]:
filtered_med_df = med_knowledge_df[med_knowledge_df['clean_infor'] != ""]
print(len(filtered_med_df))

In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

corpus = filtered_med_df['clean_infor'].values.tolist()
corpus_embeddings = embedder.encode(corpus)

In [ ]:
clustering_model = AgglomerativeClustering(n_clusters=None,
                                           linkage='average', distance_threshold=0.91)

clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

print(len(cluster_assignment))

In [ ]:
print(corpus_embeddings.shape)

In [ ]:
filtered_med_df.to_csv("./clean_medical_knowledge_df.csv", index = False)

In [1]:
!pip install openpyxl

In [4]:
import pandas as pd
df = pd.read_excel("./test_sensitive_sentences.xlsx")
df.to_csv("./test_sensitive_sentences.csv", index = False)